In [1]:
from sklearn.externals import joblib

import pandas as pd
import numpy as np

import us


In [2]:
#Load the likely voter poststratification space data

pops = pd.read_csv('demographics/pops.csv')
pops = pops.iloc[:, 1:13]

#Merge all of the District of Columbia districts into the same district number

pops.loc[pops['state'] == 'DC', 'congressional_district'] = 1

#Rename the columns to match the survey data fields

pops.columns = ['Age',
                'urbanicity',
                'Gender',
                'Area',
                'Race',
                'Education',
                'Marital Status',
                'US Congressional District',
                'What is your political party affiliation?',
                'Who will you vote for in the House of Representatives in 2018?',
                'Authoritarianism',
                'n']

#Group the poststratification space by age, gender, race, party affiliation, education, and congressional district

pops = pops.groupby(['Age',
                     'Gender',
                     'Area',
                     'Race',
                     'Education',
                     'What is your political party affiliation?',
                     'US Congressional District']).sum().reset_index()

#Rename the variable categories to match the survey data categories

pops['Gender'] = pops['Gender'].str.lower()
pops['Race'] = pops['Race'].str.lower()
pops.loc[pops['What is your political party affiliation?'] == 'Dem', 
         'What is your political party affiliation?'] = 'Democrat'
pops.loc[pops['What is your political party affiliation?'] == 'Rep',
         'What is your political party affiliation?'] = 'Republican'
pops.loc[pops['What is your political party affiliation?'] == 'Ind',
         'What is your political party affiliation?'] = 'Independent'

#Multiply the cell counts by 100 so that they represent the actual marginal population counts

pops['n'] = pops['n']*100

#Map the state abbreviations to the state names

pops['Area'] = pops['Area'].apply(lambda row: us.states.mapping('abbr', 'name')[row])

#Concatenate the state names with the district numbers to form a congressional district column

pops['US Congressional District'] = pops.apply(lambda row: row['Area'] + '-' + str(row['US Congressional District']),
                                               axis=1)


In [3]:
#Load the baseline model predictions

columns = ['US Congressional District',
           'Gender',
           'Race',
           'Age',
           'Education',
           'What is your political party affiliation?',
           'n']
major_cell_prob = pd.read_csv('baseline/major_cell_prob.csv', names=columns)
dem_cell_prob = pd.read_csv('baseline/dem_cell_prob.csv', names=columns)

# Adjust the poststratification space for all likely voters OR just Democratic voters and generate new probabilities

In [4]:
adjusted_pops = pops

adj_param = 1.2
adj_party = 'Democrat'
adj_race = 'white'
adj_gender = ''
adj_age = ''
adj_edu = ''

adjusted_pops.loc[(adjusted_pops['What is your political party affiliation?'] == adj_party) &
                                (adjusted_pops['Race'] == adj_race),
                  'n'] = adjusted_pops.loc[(adjusted_pops['What is your political party affiliation?'] == adj_party) &
                                           (adjusted_pops['Race'] == adj_race),
                                           'n']*adj_param

In [5]:
adjusted_pops.groupby([
#                      'Age',
#                      'Gender',
#                      'Area',
                     'Race',
#                      'Education',
                     'What is your political party affiliation?'
#                      'US Congressional District'
                      ]).sum().reset_index()


,Race,What is your political party affiliation?,n
0,black,Democrat,8.902492e+06
1,black,Independent,1.188286e+06
2,black,Republican,2.014121e+06
3,hispanic,Democrat,5.743063e+06
4,hispanic,Independent,1.782133e+06
5,hispanic,Republican,1.940828e+06
6,other,Democrat,3.181057e+06
7,other,Independent,1.124333e+06
8,other,Republican,1.535985e+06
9,white,Democrat,4.005437e+07


In [7]:
pops.groupby([
#                      'Age',
#                      'Gender',
#                      'Area',
                     'Race',
#                      'Education',
                     'What is your political party affiliation?'
#                      'US Congressional District'
                      ]).sum().reset_index()


,Race,What is your political party affiliation?,n
0,black,Democrat,8.902492e+06
1,black,Independent,1.188286e+06
2,black,Republican,2.014121e+06
3,hispanic,Democrat,5.743063e+06
4,hispanic,Independent,1.782133e+06
5,hispanic,Republican,1.940828e+06
6,other,Democrat,3.181057e+06
7,other,Independent,1.124333e+06
8,other,Republican,1.535985e+06
9,white,Democrat,3.371243e+07


# Obtain new vote share predictions by using the baseline/adjusted probabilities for the two models when calculating the numerator and denominator

In [6]:
N = adjusted_pops.sort_values(['US Congressional District',
                               'Gender',
                               'Race',
                               'Age',
                               'Education',
                               'What is your political party affiliation?'
                               ]).set_index(['US Congressional District',
                                             'Gender',
                                             'Race',
                                             'Age',
                                             'Education',
                                             'What is your political party affiliation?'])['n']
 
dem_cell_prob.loc[:, 'n'] = major_cell_prob.loc[:, 'n']*dem_cell_prob.loc[:, 'n']
numerator = (N*dem_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
denominator = (N*major_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
p = numerator/denominator

#Calculate the predicted Republican vote share

vote_model = pd.DataFrame(data=[p, 1 - p]).transpose().reset_index()
vote_model.columns = ['District', 'Democratic Party Vote Share', 'Republican Party Vote Share']
vote_model


,District,Democratic Party Vote Share,Republican Party Vote Share
0,Alabama-1,0.358662,0.641338
1,Alabama-2,0.405891,0.594109
2,Alabama-3,0.380113,0.619887
3,Alabama-4,0.189044,0.810956
4,Alabama-5,0.345243,0.654757
5,Alabama-6,0.271711,0.728289
6,Alabama-7,0.770573,0.229427
7,Alaska-1,0.395076,0.604924
8,Arizona-1,0.485451,0.514549
9,Arizona-2,0.533200,0.466800


In [7]:
#Find what the new balance of power would be in the House of Representatives

vote_model['Democrat Controlled'] = vote_model.apply(lambda row: 1 if row['Democratic Party Vote Share'] > 0.5 else 0,
                                                     axis=1)

#Exclude the representative from Washington DC

dem_seats = vote_model.loc[vote_model['District'] != 'District of Columbia-1', 'Democrat Controlled'].sum()

print('The Democrats won {} out of 435 districts (218 needed for a majority)'.format(dem_seats))


The Democrats won 197 out of 435 districts (218 needed for a majority)


In [9]:
#Load the likely voter poststratification space data
def adjust(adj_param,adj_party,adj_race,adj_gender,adj_age, adj_edu):
    
    pops = pd.read_csv('demographics/pops.csv')
    pops = pops.iloc[:, 1:13]

    #Merge all of the District of Columbia districts into the same district number

    pops.loc[pops['state'] == 'DC', 'congressional_district'] = 1

    #Rename the columns to match the survey data fields

    pops.columns = ['Age',
                    'urbanicity',
                    'Gender',
                    'Area',
                    'Race',
                    'Education',
                    'Marital Status',
                    'US Congressional District',
                    'What is your political party affiliation?',
                    'Who will you vote for in the House of Representatives in 2018?',
                    'Authoritarianism',
                    'n']

    #Group the poststratification space by age, gender, race, party affiliation, education, and congressional district

    pops = pops.groupby(['Age',
                         'Gender',
                         'Area',
                         'Race',
                         'Education',
                         'What is your political party affiliation?',
                         'US Congressional District']).sum().reset_index()

    #Rename the variable categories to match the survey data categories

    pops['Gender'] = pops['Gender'].str.lower()
    pops['Race'] = pops['Race'].str.lower()
    pops.loc[pops['What is your political party affiliation?'] == 'Dem', 
             'What is your political party affiliation?'] = 'Democrat'
    pops.loc[pops['What is your political party affiliation?'] == 'Rep',
             'What is your political party affiliation?'] = 'Republican'
    pops.loc[pops['What is your political party affiliation?'] == 'Ind',
             'What is your political party affiliation?'] = 'Independent'

    #Multiply the cell counts by 100 so that they represent the actual marginal population counts

    pops['n'] = pops['n']*100

    #Map the state abbreviations to the state names

    pops['Area'] = pops['Area'].apply(lambda row: us.states.mapping('abbr', 'name')[row])

    #Concatenate the state names with the district numbers to form a congressional district column

    pops['US Congressional District'] = pops.apply(lambda row: row['Area'] + '-' + str(row['US Congressional District']),
                                                   axis=1)
    columns = ['US Congressional District',
               'Gender',
               'Race',
               'Age',
               'Education',
               'What is your political party affiliation?',
               'n']
    major_cell_prob = pd.read_csv('baseline/major_cell_prob.csv', names=columns)
    dem_cell_prob = pd.read_csv('baseline/dem_cell_prob.csv', names=columns)

    adjusted_pops = pops.copy()

#     adj_param = 1.01
#     adj_party = 'Democrat'
#     adj_race = 'white'
#     adj_gender = ''
#     adj_age = ''
#     adj_edu = ''

    adjusted_pops.loc[(adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) &
                      (adjusted_pops['Gender'] == adj_gender) &
                      (adjusted_pops['Age'] == adj_age) &
                      (adjusted_pops['Education'] == adj_edu),
                      'n'] = adjusted_pops.loc[(adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) &
                      (adjusted_pops['Gender'] == adj_gender) &
                      (adjusted_pops['Age'] == adj_age) &
                      (adjusted_pops['Education'] == adj_edu),
                      'n']*adj_param

    N = adjusted_pops.sort_values(['US Congressional District',
                                   'Gender',
                                   'Race',
                                   'Age',
                                   'Education',
                                   'What is your political party affiliation?'
                                   ]).set_index(['US Congressional District',
                                                 'Gender',
                                                 'Race',
                                                 'Age',
                                                 'Education',
                                                 'What is your political party affiliation?'])['n']

    dem_cell_prob.loc[:, 'n'] = major_cell_prob.loc[:, 'n']*dem_cell_prob.loc[:, 'n']
    numerator = (N*dem_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    denominator = (N*major_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    p = numerator/denominator

    #Calculate the predicted Republican vote share

    vote_model = pd.DataFrame(data=[p, 1 - p]).transpose().reset_index()
    vote_model.columns = ['District', 'Democratic Party Vote Share', 'Republican Party Vote Share']
    # vote_model

    #Find what the new balance of power would be in the House of Representatives

    vote_model['Democrat Controlled'] = vote_model.apply(lambda row: 1 if row['Democratic Party Vote Share'] > 0.5 else 0,
                                                         axis=1)

    #Exclude the representative from Washington DC

    dem_seats = vote_model.loc[vote_model['District'] != 'District of Columbia-1', 'Democrat Controlled'].sum()

    print('If {0},{1},{2},{3},{4} turn out {5}x, the Democrats will win {6} out of 435 districts (218 needed for a majority)'.format(adj_party, adj_race, adj_gender,adj_age, adj_edu, adj_param, dem_seats))

    vote_model.to_csv('PS_adjusted/{0}{1}{2}{3}{4}{5}.csv'.format(adj_party, adj_race, adj_gender,adj_age, adj_edu, adj_param))

    return vote_model


In [12]:
adj_params = [1, 1.10, 1.25, 1.50, 3, 5]
adj_party = 'Democrat'
adj_race = 'white'
adj_gender = 'female'
adj_age = '18 - 24'
adj_edu = 'College'

for adj_param in adj_params:
    adjust(adj_param,adj_party,adj_race,adj_gender,adj_age, adj_edu)

If Democrat,white,female,18 - 24,College turn out 1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 1.25x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 1.5x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 3x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 5x, the Democrats will win 179 out of 435 districts (218 needed for a majority)


In [83]:
adj_params = [2, 5, 10]
adj_party = 'Democrat'
adj_race = 'white'
adj_gender = 'female'
adj_age = '18 - 24'
adj_edu = 'College'

for adj_param in adj_params:
    adjust(adj_param,adj_party,adj_race,adj_gender,adj_age, adj_edu)

If Democrat,white,female,18 - 24,College turn out 2x, the Democrats will win 176 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 5x, the Democrats will win 179 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 10x, the Democrats will win 184 out of 435 districts (218 needed for a majority)


In [85]:
adj_params = [1.01, 1.10, 1.25, 1.5, 2, 5, 10]
adj_party = 'Democrat'
adj_race = 'white'
adj_gender = 'female'
adj_age = '25 - 34'
adj_edu = 'College'

for adj_param in adj_params:
    adjust(adj_param,adj_party,adj_race,adj_gender,adj_age, adj_edu)

If Democrat,white,female,25 - 34,College turn out 1.01x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 1.25x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 1.5x, the Democrats will win 177 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 2x, the Democrats will win 179 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 5x, the Democrats will win 189 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 10x, the Democrats will win 202 out of 435 districts (218 needed for a majority)


In [87]:
adj_params = [1, 1.10, 1.25, 1.5, 2, 5, 10, 20, 30]
adj_parties = ['Democrat','Independent','Republican']
adj_races = ['white','black','other','hispanic']
adj_genders = ['female','male']
adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_party in adj_parties:
        for adj_race in adj_races:
            for adj_gender in adj_genders:
                for adj_age in adj_ages:
                    for adj_edu in adj_edus:
                        adjust(adj_param,adj_party,adj_race,adj_gender,adj_age, adj_edu)

If Democrat,white,female,18 - 24,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,45 - 54,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,45 - 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,> 54,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,> 54,No college turn out 1x, the Democrats will win 175 out

If Democrat,hispanic,male,> 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,18 - 24,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,18 - 24,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,25 - 34,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,25 - 34,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,45 - 54,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,45 - 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,> 54,College turn out 1x, the Demo

If Independent,hispanic,male,45 - 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,hispanic,male,> 54,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,hispanic,male,> 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,white,female,18 - 24,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,white,female,18 - 24,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,white,female,25 - 34,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,white,female,25 - 34,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,white,female,45 - 54,College turn out 1x, the Demo

If Republican,hispanic,male,25 - 34,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,hispanic,male,45 - 54,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,hispanic,male,45 - 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,hispanic,male,> 54,College turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,hispanic,male,> 54,No college turn out 1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,College turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,18 - 24,No college turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,white,female,25 - 34,College turn out 1.1x, the Democ

If Democrat,hispanic,male,18 - 24,No college turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,hispanic,male,25 - 34,College turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,hispanic,male,25 - 34,No college turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,hispanic,male,45 - 54,College turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,hispanic,male,45 - 54,No college turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,hispanic,male,> 54,College turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Democrat,hispanic,male,> 54,No college turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,white,female,18 - 24,College turn out 1.1x, the 

...

If Independent,black,female,18 - 24,College turn out 20x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,black,female,18 - 24,No college turn out 20x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,black,female,25 - 34,College turn out 20x, the Democrats will win 176 out of 435 districts (218 needed for a majority)
If Independent,black,female,25 - 34,No college turn out 20x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,black,female,45 - 54,College turn out 20x, the Democrats will win 176 out of 435 districts (218 needed for a majority)
If Independent,black,female,45 - 54,No college turn out 20x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,black,female,> 54,College turn out 20x, the Democrats will win 176 out of 435 districts (218 needed for a majority)
If Independent,black,female,> 54,No college turn out 20x

If Republican,white,male,45 - 54,No college turn out 20x, the Democrats will win 129 out of 435 districts (218 needed for a majority)
If Republican,white,male,> 54,College turn out 20x, the Democrats will win 54 out of 435 districts (218 needed for a majority)
If Republican,white,male,> 54,No college turn out 20x, the Democrats will win 60 out of 435 districts (218 needed for a majority)
If Republican,black,female,18 - 24,College turn out 20x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,black,female,18 - 24,No college turn out 20x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If Republican,black,female,25 - 34,College turn out 20x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Republican,black,female,25 - 34,No college turn out 20x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If Republican,black,female,45 - 54,College turn out 20x, the Democrats 

If Democrat,white,male,25 - 34,No college turn out 30x, the Democrats will win 235 out of 435 districts (218 needed for a majority)
If Democrat,white,male,45 - 54,College turn out 30x, the Democrats will win 274 out of 435 districts (218 needed for a majority)
If Democrat,white,male,45 - 54,No college turn out 30x, the Democrats will win 255 out of 435 districts (218 needed for a majority)
If Democrat,white,male,> 54,College turn out 30x, the Democrats will win 390 out of 435 districts (218 needed for a majority)
If Democrat,white,male,> 54,No college turn out 30x, the Democrats will win 405 out of 435 districts (218 needed for a majority)
If Democrat,black,female,18 - 24,College turn out 30x, the Democrats will win 179 out of 435 districts (218 needed for a majority)
If Democrat,black,female,18 - 24,No college turn out 30x, the Democrats will win 194 out of 435 districts (218 needed for a majority)
If Democrat,black,female,25 - 34,College turn out 30x, the Democrats will win 195 out o

If Independent,white,male,25 - 34,College turn out 30x, the Democrats will win 177 out of 435 districts (218 needed for a majority)
If Independent,white,male,25 - 34,No college turn out 30x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If Independent,white,male,45 - 54,College turn out 30x, the Democrats will win 177 out of 435 districts (218 needed for a majority)
If Independent,white,male,45 - 54,No college turn out 30x, the Democrats will win 172 out of 435 districts (218 needed for a majority)
If Independent,white,male,> 54,College turn out 30x, the Democrats will win 172 out of 435 districts (218 needed for a majority)
If Independent,white,male,> 54,No college turn out 30x, the Democrats will win 162 out of 435 districts (218 needed for a majority)
If Independent,black,female,18 - 24,College turn out 30x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If Independent,black,female,18 - 24,No college turn out 30x, the Democr

If Republican,white,female,> 54,No college turn out 30x, the Democrats will win 46 out of 435 districts (218 needed for a majority)
If Republican,white,male,18 - 24,College turn out 30x, the Democrats will win 158 out of 435 districts (218 needed for a majority)
If Republican,white,male,18 - 24,No college turn out 30x, the Democrats will win 147 out of 435 districts (218 needed for a majority)
If Republican,white,male,25 - 34,College turn out 30x, the Democrats will win 131 out of 435 districts (218 needed for a majority)
If Republican,white,male,25 - 34,No college turn out 30x, the Democrats will win 137 out of 435 districts (218 needed for a majority)
If Republican,white,male,45 - 54,College turn out 30x, the Democrats will win 91 out of 435 districts (218 needed for a majority)
If Republican,white,male,45 - 54,No college turn out 30x, the Democrats will win 109 out of 435 districts (218 needed for a majority)
If Republican,white,male,> 54,College turn out 30x, the Democrats will win

In [13]:
#Load the likely voter poststratification space data
def adjust_party_race(adj_param,adj_party,adj_race):
    
    pops = pd.read_csv('demographics/pops.csv')
    pops = pops.iloc[:, 1:13]

    #Merge all of the District of Columbia districts into the same district number

    pops.loc[pops['state'] == 'DC', 'congressional_district'] = 1

    #Rename the columns to match the survey data fields

    pops.columns = ['Age',
                    'urbanicity',
                    'Gender',
                    'Area',
                    'Race',
                    'Education',
                    'Marital Status',
                    'US Congressional District',
                    'What is your political party affiliation?',
                    'Who will you vote for in the House of Representatives in 2018?',
                    'Authoritarianism',
                    'n']

    #Group the poststratification space by age, gender, race, party affiliation, education, and congressional district

    pops = pops.groupby(['Age',
                         'Gender',
                         'Area',
                         'Race',
                         'Education',
                         'What is your political party affiliation?',
                         'US Congressional District']).sum().reset_index()

    #Rename the variable categories to match the survey data categories

    pops['Gender'] = pops['Gender'].str.lower()
    pops['Race'] = pops['Race'].str.lower()
    pops.loc[pops['What is your political party affiliation?'] == 'Dem', 
             'What is your political party affiliation?'] = 'Democrat'
    pops.loc[pops['What is your political party affiliation?'] == 'Rep',
             'What is your political party affiliation?'] = 'Republican'
    pops.loc[pops['What is your political party affiliation?'] == 'Ind',
             'What is your political party affiliation?'] = 'Independent'

    #Multiply the cell counts by 100 so that they represent the actual marginal population counts

    pops['n'] = pops['n']*100

    #Map the state abbreviations to the state names

    pops['Area'] = pops['Area'].apply(lambda row: us.states.mapping('abbr', 'name')[row])

    #Concatenate the state names with the district numbers to form a congressional district column

    pops['US Congressional District'] = pops.apply(lambda row: row['Area'] + '-' + str(row['US Congressional District']),
                                                   axis=1)
    columns = ['US Congressional District',
               'Gender',
               'Race',
               'Age',
               'Education',
               'What is your political party affiliation?',
               'n']
    major_cell_prob = pd.read_csv('baseline/major_cell_prob.csv', names=columns)
    dem_cell_prob = pd.read_csv('baseline/dem_cell_prob.csv', names=columns)

    adjusted_pops = pops.copy()

#     adj_param = 1.01
#     adj_party = 'Democrat'
#     adj_race = 'white'
#     adj_gender = ''
#     adj_age = ''
#     adj_edu = ''

    adjusted_pops.loc[(adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) ,
#                       &
#                       (adjusted_pops['Gender'] == adj_gender) &
#                       (adjusted_pops['Age'] == adj_age) &
#                       (adjusted_pops['Education'] == adj_edu),
                      'n'] = adjusted_pops.loc[(adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race),
#                                                &
#                       (adjusted_pops['Gender'] == adj_gender) &
#                       (adjusted_pops['Age'] == adj_age) &
#                       (adjusted_pops['Education'] == adj_edu),
                      'n']*adj_param

    N = adjusted_pops.sort_values(['US Congressional District',
                                   'Gender',
                                   'Race',
                                   'Age',
                                   'Education',
                                   'What is your political party affiliation?'
                                   ]).set_index(['US Congressional District',
                                                 'Gender',
                                                 'Race',
                                                 'Age',
                                                 'Education',
                                                 'What is your political party affiliation?'])['n']

    dem_cell_prob.loc[:, 'n'] = major_cell_prob.loc[:, 'n']*dem_cell_prob.loc[:, 'n']
    numerator = (N*dem_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    denominator = (N*major_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    p = numerator/denominator

    #Calculate the predicted Republican vote share

    vote_model = pd.DataFrame(data=[p, 1 - p]).transpose().reset_index()
    vote_model.columns = ['District', 'Democratic Party Vote Share', 'Republican Party Vote Share']
    # vote_model

    #Find what the new balance of power would be in the House of Representatives

    vote_model['Democrat Controlled'] = vote_model.apply(lambda row: 1 if row['Democratic Party Vote Share'] > 0.5 else 0,
                                                         axis=1)

    #Exclude the representative from Washington DC

    dem_seats = vote_model.loc[vote_model['District'] != 'District of Columbia-1', 'Democrat Controlled'].sum()

    print('If {0},{1} turn out {2}x, the Democrats will win {3} out of 435 districts (218 needed for a majority)'.format(adj_party, adj_race, adj_param, dem_seats))

    vote_model.to_csv('PS_adjusted_party_race/{0}{1}{2}.csv'.format(adj_party, adj_race, adj_param))

    return vote_model


In [31]:
adj_params = [8.96, 8.97, 8.98, 8.99]
adj_parties = ['Democrat']
adj_races = ['other']
#              ,'black','other','hispanic']
# adj_genders = ['female','male']
# adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
# adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_party in adj_parties:
        for adj_race in adj_races:
            adjust_party_race(adj_param,adj_party,adj_race)

If Democrat,other turn out 8.96x, the Democrats will win 217 out of 435 districts (218 needed for a majority)
If Democrat,other turn out 8.97x, the Democrats will win 218 out of 435 districts (218 needed for a majority)
If Democrat,other turn out 8.98x, the Democrats will win 219 out of 435 districts (218 needed for a majority)
If Democrat,other turn out 8.99x, the Democrats will win 219 out of 435 districts (218 needed for a majority)


In [111]:
adj_params = [8.8,8.9]
adj_parties = ['Democrat']
adj_races = ['other']
# adj_genders = ['female','male']
# adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
# adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_party in adj_parties:
        for adj_race in adj_races:
            adjust_party_race(adj_param,adj_party,adj_race)

If Democrat,other turn out 8.8x, the Democrats will win 217 out of 435 districts (218 needed for a majority)
If Democrat,other turn out 8.9x, the Democrats will win 218 out of 435 districts (218 needed for a majority)


In [20]:
#Load the likely voter poststratification space data
def adjust_race(adj_param,adj_race):
    
    pops = pd.read_csv('demographics/pops.csv')
    pops = pops.iloc[:, 1:13]

    #Merge all of the District of Columbia districts into the same district number

    pops.loc[pops['state'] == 'DC', 'congressional_district'] = 1

    #Rename the columns to match the survey data fields

    pops.columns = ['Age',
                    'urbanicity',
                    'Gender',
                    'Area',
                    'Race',
                    'Education',
                    'Marital Status',
                    'US Congressional District',
                    'What is your political party affiliation?',
                    'Who will you vote for in the House of Representatives in 2018?',
                    'Authoritarianism',
                    'n']

    #Group the poststratification space by age, gender, race, party affiliation, education, and congressional district

    pops = pops.groupby(['Age',
                         'Gender',
                         'Area',
                         'Race',
                         'Education',
                         'What is your political party affiliation?',
                         'US Congressional District']).sum().reset_index()

    #Rename the variable categories to match the survey data categories

    pops['Gender'] = pops['Gender'].str.lower()
    pops['Race'] = pops['Race'].str.lower()
    pops.loc[pops['What is your political party affiliation?'] == 'Dem', 
             'What is your political party affiliation?'] = 'Democrat'
    pops.loc[pops['What is your political party affiliation?'] == 'Rep',
             'What is your political party affiliation?'] = 'Republican'
    pops.loc[pops['What is your political party affiliation?'] == 'Ind',
             'What is your political party affiliation?'] = 'Independent'

    #Multiply the cell counts by 100 so that they represent the actual marginal population counts

    pops['n'] = pops['n']*100

    #Map the state abbreviations to the state names

    pops['Area'] = pops['Area'].apply(lambda row: us.states.mapping('abbr', 'name')[row])

    #Concatenate the state names with the district numbers to form a congressional district column

    pops['US Congressional District'] = pops.apply(lambda row: row['Area'] + '-' + str(row['US Congressional District']),
                                                   axis=1)
    columns = ['US Congressional District',
               'Gender',
               'Race',
               'Age',
               'Education',
               'What is your political party affiliation?',
               'n']
    major_cell_prob = pd.read_csv('baseline/major_cell_prob.csv', names=columns)
    dem_cell_prob = pd.read_csv('baseline/dem_cell_prob.csv', names=columns)

    adjusted_pops = pops.copy()

#     adj_param = 1.01
#     adj_party = 'Democrat'
#     adj_race = 'white'
#     adj_gender = ''
#     adj_age = ''
#     adj_edu = ''

    adjusted_pops.loc[
#         (adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) ,
#         &
#                       (adjusted_pops['Gender'] == adj_gender) &
#                       (adjusted_pops['Age'] == adj_age) ,
#                       &
#                       (adjusted_pops['Education'] == adj_edu),
                      'n'] = adjusted_pops.loc[
#         (adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) ,
#         &
#                       (adjusted_pops['Gender'] == adj_gender) &
#                       (adjusted_pops['Age'] == adj_age) ,
#                                                &
#                       (adjusted_pops['Education'] == adj_edu),
                      'n']*adj_param

    N = adjusted_pops.sort_values(['US Congressional District',
                                   'Gender',
                                   'Race',
                                   'Age',
                                   'Education',
                                   'What is your political party affiliation?'
                                   ]).set_index(['US Congressional District',
                                                 'Gender',
                                                 'Race',
                                                 'Age',
                                                 'Education',
                                                 'What is your political party affiliation?'])['n']

    dem_cell_prob.loc[:, 'n'] = major_cell_prob.loc[:, 'n']*dem_cell_prob.loc[:, 'n']
    numerator = (N*dem_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    denominator = (N*major_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    p = numerator/denominator

    #Calculate the predicted Republican vote share

    vote_model = pd.DataFrame(data=[p, 1 - p]).transpose().reset_index()
    vote_model.columns = ['District', 'Democratic Party Vote Share', 'Republican Party Vote Share']
    # vote_model

    #Find what the new balance of power would be in the House of Representatives

    vote_model['Democrat Controlled'] = vote_model.apply(lambda row: 1 if row['Democratic Party Vote Share'] > 0.5 else 0,
                                                         axis=1)

    #Exclude the representative from Washington DC

    dem_seats = vote_model.loc[vote_model['District'] != 'District of Columbia-1', 'Democrat Controlled'].sum()

    print('If {0} turn out {1}x, the Democrats will win {2} out of 435 districts (218 needed for a majority)'.format(adj_race,adj_param, dem_seats))

    vote_model.to_csv('PS_adjusted_race/{0}{1}.csv'.format(adj_race,adj_param))

    return vote_model


In [23]:
adj_params = [1.10, 1.25, 1.5, 2, 5, 10]
# adj_parties = ['Democrat']
# adj_races = ['white','black','other','hispanic']
adj_genders = ['female','male']
adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_gender in adj_races:
        adjust_race(adj_param,adj_race)

If white turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If black turn out 1.1x, the Democrats will win 176 out of 435 districts (218 needed for a majority)
If other turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If hispanic turn out 1.1x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If white turn out 1.25x, the Democrats will win 171 out of 435 districts (218 needed for a majority)
If black turn out 1.25x, the Democrats will win 178 out of 435 districts (218 needed for a majority)
If other turn out 1.25x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If hispanic turn out 1.25x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If white turn out 1.5x, the Democrats will win 169 out of 435 districts (218 needed for a majority)
If black turn out 1.5x, the Democrats will win 184 out of 435 districts (218 needed for a 

In [ ]:
adj_params = [1.10, 1.25, 1.5, 2, 5, 10]
# adj_parties = ['Democrat']
adj_races = ['white','black','other','hispanic']
# adj_genders = ['female','male']
# adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
# adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_race in adj_races:
        adjust_race(adj_param,adj_race)

In [8]:
#Load the likely voter poststratification space data
def adjust_gender_age_edu(adj_param,adj_gender, adj_age, adj_edu):
    
    pops = pd.read_csv('demographics/pops.csv')
    pops = pops.iloc[:, 1:13]

    #Merge all of the District of Columbia districts into the same district number

    pops.loc[pops['state'] == 'DC', 'congressional_district'] = 1

    #Rename the columns to match the survey data fields

    pops.columns = ['Age',
                    'urbanicity',
                    'Gender',
                    'Area',
                    'Race',
                    'Education',
                    'Marital Status',
                    'US Congressional District',
                    'What is your political party affiliation?',
                    'Who will you vote for in the House of Representatives in 2018?',
                    'Authoritarianism',
                    'n']

    #Group the poststratification space by age, gender, race, party affiliation, education, and congressional district

    pops = pops.groupby(['Age',
                         'Gender',
                         'Area',
                         'Race',
                         'Education',
                         'What is your political party affiliation?',
                         'US Congressional District']).sum().reset_index()

    #Rename the variable categories to match the survey data categories

    pops['Gender'] = pops['Gender'].str.lower()
    pops['Race'] = pops['Race'].str.lower()
    pops.loc[pops['What is your political party affiliation?'] == 'Dem', 
             'What is your political party affiliation?'] = 'Democrat'
    pops.loc[pops['What is your political party affiliation?'] == 'Rep',
             'What is your political party affiliation?'] = 'Republican'
    pops.loc[pops['What is your political party affiliation?'] == 'Ind',
             'What is your political party affiliation?'] = 'Independent'

    #Multiply the cell counts by 100 so that they represent the actual marginal population counts

    pops['n'] = pops['n']*100

    #Map the state abbreviations to the state names

    pops['Area'] = pops['Area'].apply(lambda row: us.states.mapping('abbr', 'name')[row])

    #Concatenate the state names with the district numbers to form a congressional district column

    pops['US Congressional District'] = pops.apply(lambda row: row['Area'] + '-' + str(row['US Congressional District']),
                                                   axis=1)
    columns = ['US Congressional District',
               'Gender',
               'Race',
               'Age',
               'Education',
               'What is your political party affiliation?',
               'n']
    major_cell_prob = pd.read_csv('baseline/major_cell_prob.csv', names=columns)
    dem_cell_prob = pd.read_csv('baseline/dem_cell_prob.csv', names=columns)

    adjusted_pops = pops.copy()

#     adj_param = 1.01
#     adj_party = 'Democrat'
#     adj_race = 'white'
#     adj_gender = ''
#     adj_age = ''
#     adj_edu = ''

    adjusted_pops.loc[
#         (adjusted_pops['What is your political party affiliation?'] == adj_party) &
#                       (adjusted_pops['Race'] == adj_race) ,
#         &
                      (adjusted_pops['Gender'] == adj_gender) &
                      (adjusted_pops['Age'] == adj_age) &
                      (adjusted_pops['Education'] == adj_edu),
                      'n'] = adjusted_pops.loc[
#         (adjusted_pops['What is your political party affiliation?'] == adj_party) &
#                       (adjusted_pops['Race'] == adj_race) ,
#         &
                      (adjusted_pops['Gender'] == adj_gender) &
                      (adjusted_pops['Age'] == adj_age) &
                      (adjusted_pops['Education'] == adj_edu),
                      'n']*adj_param

    N = adjusted_pops.sort_values(['US Congressional District',
                                   'Gender',
                                   'Race',
                                   'Age',
                                   'Education',
                                   'What is your political party affiliation?'
                                   ]).set_index(['US Congressional District',
                                                 'Gender',
                                                 'Race',
                                                 'Age',
                                                 'Education',
                                                 'What is your political party affiliation?'])['n']

    dem_cell_prob.loc[:, 'n'] = major_cell_prob.loc[:, 'n']*dem_cell_prob.loc[:, 'n']
    numerator = (N*dem_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    denominator = (N*major_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    p = numerator/denominator

    #Calculate the predicted Republican vote share

    vote_model = pd.DataFrame(data=[p, 1 - p]).transpose().reset_index()
    vote_model.columns = ['District', 'Democratic Party Vote Share', 'Republican Party Vote Share']
    # vote_model

    #Find what the new balance of power would be in the House of Representatives

    vote_model['Democrat Controlled'] = vote_model.apply(lambda row: 1 if row['Democratic Party Vote Share'] > 0.5 else 0,
                                                         axis=1)

    #Exclude the representative from Washington DC

    dem_seats = vote_model.loc[vote_model['District'] != 'District of Columbia-1', 'Democrat Controlled'].sum()

    print('If {0}{1}{2} turn out {3}x, the Democrats will win {4} out of 435 districts (218 needed for a majority)'.format(adj_gender, adj_edu, adj_age,adj_param, dem_seats))

    vote_model.to_csv('PS_adjusted_race/{0}{1}{2}{3}.csv'.format(adj_gender, adj_edu, adj_age,adj_param))

    return vote_model


In [9]:
adj_params = [1.10, 1.25, 1.5, 2]
# adj_parties = ['Democrat']
# adj_races = ['white','black','other','hispanic']
adj_genders = ['female','male']
adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_gender in adj_genders:
        for adj_age in adj_ages:
            for adj_edu in adj_edus:
                adjust_gender_age_edu(adj_param,adj_gender, adj_age, adj_edu)

If femaleCollege18 - 24 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleNo college18 - 24 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleCollege25 - 34 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleNo college25 - 34 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleCollege45 - 54 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleNo college45 - 54 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleCollege> 54 turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If femaleNo college> 54 turn out 1.1x, the Democrats will win 174 out of 435 districts (218 needed for a majority)


KeyboardInterrupt: 

In [10]:
#Load the likely voter poststratification space data
def adjust_gender_race(adj_param,adj_gender, adj_race):
    
    pops = pd.read_csv('demographics/pops.csv')
    pops = pops.iloc[:, 1:13]

    #Merge all of the District of Columbia districts into the same district number

    pops.loc[pops['state'] == 'DC', 'congressional_district'] = 1

    #Rename the columns to match the survey data fields

    pops.columns = ['Age',
                    'urbanicity',
                    'Gender',
                    'Area',
                    'Race',
                    'Education',
                    'Marital Status',
                    'US Congressional District',
                    'What is your political party affiliation?',
                    'Who will you vote for in the House of Representatives in 2018?',
                    'Authoritarianism',
                    'n']

    #Group the poststratification space by age, gender, race, party affiliation, education, and congressional district

    pops = pops.groupby(['Age',
                         'Gender',
                         'Area',
                         'Race',
                         'Education',
                         'What is your political party affiliation?',
                         'US Congressional District']).sum().reset_index()

    #Rename the variable categories to match the survey data categories

    pops['Gender'] = pops['Gender'].str.lower()
    pops['Race'] = pops['Race'].str.lower()
    pops.loc[pops['What is your political party affiliation?'] == 'Dem', 
             'What is your political party affiliation?'] = 'Democrat'
    pops.loc[pops['What is your political party affiliation?'] == 'Rep',
             'What is your political party affiliation?'] = 'Republican'
    pops.loc[pops['What is your political party affiliation?'] == 'Ind',
             'What is your political party affiliation?'] = 'Independent'

    #Multiply the cell counts by 100 so that they represent the actual marginal population counts

    pops['n'] = pops['n']*100

    #Map the state abbreviations to the state names

    pops['Area'] = pops['Area'].apply(lambda row: us.states.mapping('abbr', 'name')[row])

    #Concatenate the state names with the district numbers to form a congressional district column

    pops['US Congressional District'] = pops.apply(lambda row: row['Area'] + '-' + str(row['US Congressional District']),
                                                   axis=1)
    columns = ['US Congressional District',
               'Gender',
               'Race',
               'Age',
               'Education',
               'What is your political party affiliation?',
               'n']
    major_cell_prob = pd.read_csv('baseline/major_cell_prob.csv', names=columns)
    dem_cell_prob = pd.read_csv('baseline/dem_cell_prob.csv', names=columns)

    adjusted_pops = pops.copy()

#     adj_param = 1.01
#     adj_party = 'Democrat'
#     adj_race = 'white'
#     adj_gender = ''
#     adj_age = ''
#     adj_edu = ''

    adjusted_pops.loc[
#         (adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) &
                      (adjusted_pops['Gender'] == adj_gender) ,
#                       (adjusted_pops['Age'] == adj_age) &
#                       (adjusted_pops['Education'] == adj_edu),
                      'n'] = adjusted_pops.loc[
#         (adjusted_pops['What is your political party affiliation?'] == adj_party) &
                      (adjusted_pops['Race'] == adj_race) &
                      (adjusted_pops['Gender'] == adj_gender) ,
#                       (adjusted_pops['Age'] == adj_age) &
#                       (adjusted_pops['Education'] == adj_edu),
                      'n']*adj_param

    N = adjusted_pops.sort_values(['US Congressional District',
                                   'Gender',
                                   'Race',
                                   'Age',
                                   'Education',
                                   'What is your political party affiliation?'
                                   ]).set_index(['US Congressional District',
                                                 'Gender',
                                                 'Race',
                                                 'Age',
                                                 'Education',
                                                 'What is your political party affiliation?'])['n']

    dem_cell_prob.loc[:, 'n'] = major_cell_prob.loc[:, 'n']*dem_cell_prob.loc[:, 'n']
    numerator = (N*dem_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    denominator = (N*major_cell_prob.loc[:, 'n'].values).groupby(['US Congressional District']).sum()
    p = numerator/denominator

    #Calculate the predicted Republican vote share

    vote_model = pd.DataFrame(data=[p, 1 - p]).transpose().reset_index()
    vote_model.columns = ['District', 'Democratic Party Vote Share', 'Republican Party Vote Share']
    # vote_model

    #Find what the new balance of power would be in the House of Representatives

    vote_model['Democrat Controlled'] = vote_model.apply(lambda row: 1 if row['Democratic Party Vote Share'] > 0.5 else 0,
                                                         axis=1)

    #Exclude the representative from Washington DC

    dem_seats = vote_model.loc[vote_model['District'] != 'District of Columbia-1', 'Democrat Controlled'].sum()

    print('If {0} {1} turn out {2}x, the Democrats will win {3} out of 435 districts (218 needed for a majority)'.format(adj_gender, adj_race,adj_param, dem_seats))

    vote_model.to_csv('PS_adjusted_race/{0}{1}{2}.csv'.format(adj_gender, adj_race,adj_param))

    return vote_model


In [11]:
adj_params = [1.10, 1.25, 1.5, 2]
# adj_parties = ['Democrat']
adj_races = ['white','black','other','hispanic']
adj_genders = ['female','male']
# adj_ages = ['18 - 24','25 - 34','45 - 54','> 54']
# adj_edus = ['College','No college']

for adj_param in adj_params:
    for adj_gender in adj_genders:
        for adj_race in adj_races:
            adjust_gender_race(adj_param,adj_gender, adj_race)

If female white turn out 1.1x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If female black turn out 1.1x, the Democrats will win 175 out of 435 districts (218 needed for a majority)
If female other turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If female hispanic turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If male white turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If male black turn out 1.1x, the Democrats will win 174 out of 435 districts (218 needed for a majority)
If male other turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If male hispanic turn out 1.1x, the Democrats will win 173 out of 435 districts (218 needed for a majority)
If female white turn out 1.25x, the Democrats will win 176 out of 435 districts (218 needed for a majority)
If female black turn out 1.25x, the De